In [1]:
import os

# 읽은 파일 내용을 저장할 빈 리스트 생성
poker_hand_data = []

# 파일이 저장된 디렉토리 경로 설정
directory_path = 'E:\chatGPT\Holdem\HandData'

# 디렉토리 내의 모든 파일에 대해 반복
for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):  # 파일 확장자에 따라 필터링
        file_path = os.path.join(directory_path, filename)
        
        # 파일을 읽어서 리스트에 추가
        with open(file_path, 'r') as file:
            file_contents = file.read()
            poker_hand_data.append(file_contents)

hand_data_list = []

# 핸드데이터 정리
for poker_hand in poker_hand_data:
    hand_data = [line.strip() for line in poker_hand.strip().split('\n\n')]

    hand_data_list = hand_data_list + hand_data

len(hand_data_list)

2598

In [2]:
# 포커 핸드 데이터를 줄 단위로 분할
hand_data_list = [line.strip() for line in poker_hand_data.strip().split('\n\n')]

AttributeError: 'list' object has no attribute 'strip'

In [36]:
import pandas as pd
import re

# DataFrame을 저장할 리스트
hero_hands = []

# 각 포커 핸드 데이터에 대한 반복
for hand_data in hand_data_list:
    hand_date = hand_data.split('\n')[0]
    
    # 날짜를 추출하는 정규 표현식 패턴
    date_pattern = r"\d{4}/\d{2}/\d{2} \d{2}:\d{2}:\d{2}"
    
    match = re.search(date_pattern, hand_date)
    if match:
        extracted_date = match.group()
        
        for line in hand_data.split('\n'):
            if "Dealt to Hero" in line:
                hero_hand = line.split('[')[1].split(']')[0]
                # 날짜와 hero_hand 값을 리스트에 추가
                hero_hands.append([extracted_date, hero_hand])

# DataFrame 생성
df = pd.DataFrame(hero_hands, columns=['Date', 'Hero Hand'])

# 'Date' 열을 datetime 자료형으로 변환
df['Date'] = pd.to_datetime(df['Date'])

# 날짜를 기준으로 DataFrame을 정렬
df = df.sort_values(by='Date')

# 정렬된 DataFrame 출력
print(df)


                    Date Hero Hand
221  2023-12-21 23:01:59     2c 9c
449  2023-12-21 23:02:00     Qh 5d
220  2023-12-21 23:02:08     9c Qs
448  2023-12-21 23:02:10     Jc 8h
219  2023-12-21 23:02:16     5c 6d
...                  ...       ...
2109 2023-12-27 22:15:24     4h Jh
2108 2023-12-27 22:15:28     5c 2d
2107 2023-12-27 22:15:37     Jh 3d
2106 2023-12-27 22:15:43     3d Qh
2105 2023-12-27 22:15:47     As Ad

[2598 rows x 2 columns]


In [38]:
import numpy as np

HoleCardsOdds = np.genfromtxt('HoleCardsOdds.csv', delimiter=',', dtype=None, encoding='utf-8')


In [39]:
# 초기화된 배열 생성
HoleCards = np.zeros((13, 13), dtype=int)

rank_mapping = {'A': 0, 'K': 1, 'Q': 2, 'J': 3, 'T': 4, '9': 5, '8': 6, '7': 7, '6': 8, '5': 9, '4': 10, '3': 11, '2': 12}

hero_hands = df['Hero Hand']

for hand in hero_hands:
    cards = hand.split() # cards = ['As', 'Ad']
    
    rank1, rank2 = cards[0][0], cards[1][0]
    suit1, suit2 = cards[0][1], cards[1][1]

    if suit1 == suit2:
        HoleCards[min(rank_mapping[rank1], rank_mapping[rank2]), max(rank_mapping[rank1], rank_mapping[rank2])] += 1
    else:
        HoleCards[max(rank_mapping[rank1], rank_mapping[rank2]), min(rank_mapping[rank1], rank_mapping[rank2])] += 1


# 
HoleCards = np.divide(HoleCards, np.sum(HoleCards))

# 두 배열 간의 차이 계산
diff = HoleCardsOdds - HoleCards

# 차이의 제곱 계산
squared_diff = diff ** 2

# 차이의 평균 계산
mean_diff = np.mean(squared_diff)

# 평균의 제곱근을 계산하여 표준 오차 구하기
std_error = np.sqrt(mean_diff)

# 결과 출력
print("표준 오차:", std_error)


표준 오차: 0.0015339538413279086
